In [12]:
import dlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from sklearn.linear_model import LogisticRegressionCV

In [9]:
predictor_path = "./shape_predictor_68_face_landmarks.dat"
face_rec_model_path = "./dlib_face_recognition_resnet_model_v1.dat"
 
detector = dlib.get_frontal_face_detector() #a detector to find the faces
sp = dlib.shape_predictor(predictor_path ) #shape predictor to find face landmarks
facerec = dlib.face_recognition_model_v1(face_rec_model_path) #face recognition model
img_data=np.zeros((5000,128))

In [1]:
for i in range (5000):
    img = dlib.load_rgb_image('../datasets/celeba/img/' + str(i) + '.jpg')
    dets = detector(img, 1)  #Extract the face area in the picture
    for k, d in enumerate(dets):
        shape = sp(img, d)
        face_descriptor = facerec.compute_face_descriptor(img, shape)
        img_data[i,:]=face_descriptor

NameError: name 'dlib' is not defined

In [18]:
label=pd.read_table('../datasets/celeba/labels.csv')

In [20]:
x = pd.read_csv('./face feature.csv')
x=np.array(x)
x=x[:,1:]
print(x)
print(np.shape(x))

[[-0.11575226  0.13944775  0.04134562 ... -0.06901028  0.08327678
   0.00861722]
 [-0.11231291  0.03962129  0.04105864 ... -0.00192231  0.02313073
  -0.01509616]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.00168826 -0.0081973   0.00677922 ... -0.0436994   0.09507341
   0.0518407 ]
 [-0.14209679  0.04581597  0.08493239 ... -0.07254556  0.02568161
  -0.00020073]
 [-0.04625478  0.14262904  0.11789934 ... -0.09501354  0.06680886
   0.04954072]]
(5000, 128)


In [21]:
# Split the data
#x=img_data
y = label['smiling']

# Split the data into training and testing(75% training and 25% testing data)
x_train, x_test, y_train, y_test = train_test_split(x, y,random_state=0)# Pre-process data
from sklearn.preprocessing import StandardScaler
x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)

In [22]:
logreg = LogisticRegression(solver='lbfgs',max_iter=1000)
logreg.fit(x_train, y_train)
y_pred= logreg.predict(x_test)
print('Accuracy on train set:'+str(logreg.score(x_train,y_train)))
print('Accuracy on test set: '+str(accuracy_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))#text report showing the main classification metrics

Accuracy on train set:0.6624
Accuracy on test set: 0.6256
              precision    recall  f1-score   support

          -1       0.64      0.61      0.62       639
           1       0.61      0.64      0.63       611

    accuracy                           0.63      1250
   macro avg       0.63      0.63      0.63      1250
weighted avg       0.63      0.63      0.63      1250



In [23]:
import pickle


pickle.dump(logreg,open("LogisticRegression_smiling_dlib.dat","wb")) 


loaded_model = pickle.load(open("LogisticRegression_smiling_dlib.dat","rb"))


y_pred=loaded_model.predict(x_test)